In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import yaml
import pathlib
from types import SimpleNamespace # Used to mimic argparse.Namespace

In [3]:
# --- Configuration Loading ---
def recursive_update(base, update):
    """Recursively updates a dictionary `base` with values from `update`."""
    for key, value in update.items():
        if isinstance(value, dict) and key in base:
            recursive_update(base[key], value)
        else:
            base[key] = value

def load_configs(config_names=None):
    """
    Loads configurations from 'configs.yaml', applying defaults and
    specified overrides.
    """
    # Adjust this path if configs.yaml is not in the same directory
    config_path = pathlib.Path("~/dreamerv3-torch/configs.yaml").expanduser()
    
    if not config_path.exists():
        raise FileNotFoundError(f"configs.yaml not found at {config_path}. "
                                "Please create it or adjust the path.")

    all_configs = yaml.safe_load(config_path.read_text())

    name_list = ["defaults", *config_names] if config_names else ["defaults"]
    
    final_config = {}
    for name in name_list:
        if name not in all_configs:
            print(f"Warning: Configuration '{name}' not found in configs.yaml. Skipping.")
            continue
        recursive_update(final_config, all_configs[name])
    
    # Convert the dictionary to a SimpleNamespace to mimic argparse.Namespace
    return SimpleNamespace(**final_config)



# --- Mimicking Command Line Arguments (for Jupyter) ---
config = load_configs(config_names=['maniskill'])
print(config.task, config.units)

maniskill_PickCube-v1 512


In [4]:
from dreamer import make_env
env = make_env(config, mode="train", id=0)


/home/jstaley_theaiinstitute_com/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'NoneType' object has no attribute 'glGetError'

In [ ]:
print("Observation space", env.observation_space)
print("Action space", env.action_space)

# obs, _ = env.reset(seed=0) # TODO: reset with a seed for determinism
obs, _ = env.reset() # TODO: reset with a seed for determinism


In [ ]:

done = False
while not done:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step({'action': action})
    done = terminated or truncated
    env.env.env.env.env.render() # maniskill render doesn't take any arguments, but the gymnasium environment does. Annoying.
env.close()
